# Benchmark modelos de Fasttext(BIO) (Subwords) para obtener Embeddings del texto

In [ ]:
! python -m spacy download es_core_news_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-27 10:46:27.035359: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
! pip install spacy
#! python -m spacy download es
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def NLP_preprocesaDocumento(nlp, doc):

  #Convertir a minúsculas
  doc = doc.lower()
  
  #Tokenizado
  doc = [" ".join([token.text for token in nlp(doc)])][0]

  #Eliminación de signos de puntuación
  doc = [" ".join([token.lemma_ for token in nlp(doc) if not token.is_punct]) ][0]

  #Eliminación de stopwords
  spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
  doc = [" ".join([token for token in doc.split() if not token.lower() in spacy_stopwords])][0]
  
  #Lematización de palabras
  doc = [" ".join([token.lemma_ for token in nlp(doc)])][0]

  #Eliminación de tildes
  tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'ü': 'u', 'Ü': 'U'}
  for tilde in tildes:
	  if tilde in doc:
		  doc = doc.replace(tilde, tildes[tilde])

  return doc


In [ ]:
def split_sentences(nlp, text):
    tokens = nlp(text)
    sentences = []
    for sent in tokens.sents:
        sentences.append(sent.text.strip())
    return sentences

In [ ]:
#Cargamos el modelo en español
nlp_lg = spacy.load("es_core_news_lg")

## Carga de las terminologías clínicas SNOMED-CT y generación de sus Embeddings
NOTA: Estos solo es necesario hacerlo la primera vez

In [ ]:
import os
def cargaProblemasSaludAHSpainExtension():
  f = open('der2_Refset_ProblemasSaludAHSpainExtensionSnapshot_ES_20221201.txt')
  terminos_snomedct_problemassaludAH = []
  cont = 0
  for row in f:
    if cont>0:
      values = row.strip().split("\t")
      #id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
      #print(values)
      termino = {}
      termino["id"] = values[0];
      #termino["effectiveTime"] = values[1];
      #termino["active"] = values[2];
      #termino["moduleId"] = values[3];
      #termino["refsetId"] = values[4];
      #termino["referencedComponentId"] = values[5];
      termino["term"] = values[6];
      #termino["num_palabras"] = len(values[6].split(" "));
      #print(termino)
      terminos_snomedct_problemassaludAH.append(termino)
    cont = cont + 1

  f.close();
  return terminos_snomedct_problemassaludAH;

In [ ]:
!pip install sentencepiece
import sentencepiece as spm

sp = spm.SentencePieceProcessor(model_file='drive/MyDrive/models/sp-bio-sw-emb-uncased-bio.model')

def anadirEmbedding(model,nlp,lista,num_items_max=0,preprocesado=False):
  cont = 0
  for item in lista:
    #print(item["term"])
    if preprocesado:
      doc = NLP_preprocesaDocumento(nlp,item["term"])
      if doc == "":
        print("ERROR AL Parsear " + item["term"])
        doc = item["term"]
    else:
      doc = item["term"]
    #print(doc)
    #item["embedding"] = model.wv.get_vector(doc)
    item["embedding"] = model.wv.get_vector(sp.decode([sp.bos_id()] + sp.encode_as_ids(doc) + [sp.eos_id()]))
    cont = cont + 1
    if cont == num_items_max:
      break
  return lista

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
terminos_snomedct_problemassaludAH = cargaProblemasSaludAHSpainExtension();

In [ ]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

#numpyArrayOne = numpy.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]])

# Serialization
#numpyData = {"array": numpyArrayOne}
#encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
#print("Printing JSON serialized NumPy array")
#print(encodedNumpyData)

# Deserialization
#print("Decode JSON serialized NumPy array")
#decodedArrays = json.loads(encodedNumpyData)

#finalNumpyArray = numpy.asarray(decodedArrays["array"])
#print("NumPy Array")
#print(finalNumpyArray)

In [ ]:
!pip install gensim
from gensim.models import FastText
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Obtención de embeddings con el modelo seleccionado (nlp_lg y model sp-cli-w-emb-uncased-cbow-d50-clinic_es)

In [ ]:
#Tiempo de ejecucion: 3 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-cbow-d50-bio_es.bin')

In [ ]:
#Tiempo de ejecucion: 56 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_sp-bio-sw-emb-uncased-cbow-d50-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 7 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-cbow-d100-bio_es.bin')

In [ ]:
#Tiempo de ejecucion: 58 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_sp-bio-sw-emb-uncased-cbow-d100-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 21 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-cbow-d300-bio_es.bin')

In [ ]:
#Tiempo de ejecucion: 55 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_sp-bio-sw-emb-uncased-cbow-d300-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 4 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-skip-d50-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 51 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);


ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)
with open('catalogo_sp-bio-sw-emb-uncased-skip-d50-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 7 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-skip-d100-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 52 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);


ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)
with open('catalogo_sp-bio-sw-emb-uncased-skip-d100-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 16 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-skip-d300-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 53 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);


ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)
with open('catalogo_sp-bio-sw-emb-uncased-skip-d300-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

## Obtención de embeddings, carga de modelos y pruebas

In [ ]:
!pip install gensim
from gensim.models import FastText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def obtenEmbedding(model, doc):
  return model.wv.get_vector(sp.decode([sp.bos_id()] + sp.encode_as_ids(doc) + [sp.eos_id()]))

In [ ]:
#Tiempo de ejecucion: 24 sg
model_cc50 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-cbow-d50-bio_es.bin')
model_cc100 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-cbow-d100-bio_es.bin')
model_cc300 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-cbow-d300-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 22 sg
model_cs50 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-skip-d50-bio_es.bin')
model_cs100 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-skip-d100-bio_es.bin')
model_cs300 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-sw-emb-uncased-skip-d300-bio_es.bin')

In [ ]:
import os, json, numpy

f = open('catalogo_sp-bio-sw-emb-uncased-cbow-d50-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cc50 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cc50.append(termino)
f.close();

f = open('catalogo_sp-bio-sw-emb-uncased-cbow-d100-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cc100 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cc100.append(termino)
f.close();

f = open('catalogo_sp-bio-sw-emb-uncased-cbow-d300-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cc300 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cc300.append(termino)
f.close();

In [ ]:
f = open('catalogo_sp-bio-sw-emb-uncased-skip-d50-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cs50 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cs50.append(termino)
f.close();

f = open('catalogo_sp-bio-sw-emb-uncased-skip-d100-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cs100 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cs100.append(termino)
f.close();

f = open('catalogo_sp-bio-sw-emb-uncased-skip-d300-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cs300 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cs300.append(termino)
f.close();

In [ ]:
def NLP_get_ngrams(text, n):
    text = text.split()
    output = []
    for i in range(len(text) - n + 1):
        output.append(text[i:i+n])
    return output

In [ ]:
#informe = "El paciente presenta el abdomen distenido ."
#informe = "El paciente presenta un aborto espontáneo y retenido, distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
#informe = "El paciente presenta un abdomen distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."


In [ ]:
import numpy as np
from numpy.linalg import norm
from operator import itemgetter

def get_terminos_vinculados(model, catalog, similarity_threshold, similarity, informe):
  terminos_vinculados = []

  frases = split_sentences(nlp_lg, informe)
  for frase in frases:
    informe_preprocesado = NLP_preprocesaDocumento(nlp_lg, frase)

    #Empezamos por ngramas más grandes a más pequeños
    for num_ngrams in range(6, 0, -1):
      ngramas = NLP_get_ngrams(informe_preprocesado, num_ngrams)
      for ngrama in ngramas:
        doc = " ".join([token for token in ngrama])
        #print(doc)
        doc_embedding = model.wv.get_vector(doc)
        vector_A = doc_embedding
        #Por cada término de la terminología, comprobamos la similaridad con el ngrama previo
        terminos_candidatos = []
        for termino in catalog:
          vector_B = termino["embedding"]
          if similarity == 0:
            dist = numpy.linalg.norm(vector_A-vector_B)
            if dist <= similarity_threshold:
              #print(doc, " ", termino["term"], " Similarity:", dist)
              terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "similarity": dist})
            #Nos quedamos con el término con menor distancia de los obtenidos
            if terminos_candidatos:
              terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('similarity'), reverse=False) 
              terminos_vinculados.append(terminos_candidatos_sorted[0])
          else:
            dist = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
            if dist >= similarity_threshold:
              #print(doc, " ", termino["term"], " Similarity:", dist)
              terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "similarity": dist})
            #Nos quedamos con el término con mayor probabilidad de los obtenidos
            if terminos_candidatos:
              terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('similarity'), reverse=True) 
              terminos_vinculados.append(terminos_candidatos_sorted[0])

  print("Resultado:")

  if similarity == 0:
    #Eliminamos duplicados
    terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
    termino_ant = {"term":"", "similarity": "100.0"}
    terminos_vinculados_deduplicated = []
    for termino in terminos_vinculados_sorted:
      if termino["term"] != termino_ant["term"]:
        if termino_ant["term"] != "":
          terminos_vinculados_deduplicated.append(termino_ant)
          #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])
        termino_ant = termino
      else:
        if termino_ant["similarity"]>termino["similarity"]:
          termino_ant = termino
    terminos_vinculados_deduplicated.append(termino_ant)
    #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])

  else:
    #Eliminamos duplicados
    terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
    termino_ant = {"term":"", "similarity": "0.0"}
    terminos_vinculados_deduplicated = []
    for termino in terminos_vinculados_sorted:
      if termino["term"] != termino_ant["term"]:
        if termino_ant["term"] != "":
          terminos_vinculados_deduplicated.append(termino_ant)
          #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])
        termino_ant = termino
      else:
        if termino_ant["similarity"]<termino["similarity"]:
          termino_ant = termino
    terminos_vinculados_deduplicated.append(termino_ant)
    #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])

  return terminos_vinculados_deduplicated


In [ ]:
#Tiempo de ejecucion: 10 sg
terminos_vinculados = get_terminos_vinculados(model_cc50, terminos_snomedct_problemassaludAH_cc50, 10, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])


Resultado:
absceso abdominal  Similarity: 4.546358087231224
accidentes producidos por instrumentos u objetos cortantes y para piercing  Similarity: 8.39342105566491
adherencias intestinales con obstrucción  Similarity: 5.547472546957855
agresión por instrumentos cortantes y punzantes  Similarity: 8.417195600715775
alteración visual moderada  Similarity: 9.291776864507915
anemia megaloblástica por deficiencia de ácido fólico  Similarity: 8.93816188421902
aneurisma de aorta abdominal  Similarity: 9.169241336549158
aneurisma de aorta abdominal en aumento  Similarity: 9.790220729535214
anomalía fetal sospechada que afecta el manejo de la madre  Similarity: 9.897977596942182
anticoncepción con dispositivo intrauterino  Similarity: 9.30172572033451
apendicitis  Similarity: 0.0
aplastamiento vertebral  Similarity: 9.049364842313246
arteriosclerosis obliterante  Similarity: 8.814866272531953
artritis de articulación esternoclavicular  Similarity: 9.48217230417043
artritis de articulación ester

In [ ]:
#Tiempo de ejecucion: 19 sg
terminos_vinculados = get_terminos_vinculados(model_cc50, terminos_snomedct_problemassaludAH_cc50, 0.85, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Cosine Similarity: 0.9665122975625563
adherencias intestinales con obstrucción  Cosine Similarity: 0.9011206416597514
anemia hemolítica  Cosine Similarity: 0.9036948652678766
apendicitis  Cosine Similarity: 1.000000014706098
ascitis  Cosine Similarity: 1.0000000613743327
asma sin estado asmático  Cosine Similarity: 0.8682262506489348
aterosclerosis aortoilíaca  Cosine Similarity: 0.8799879307479621
aterosclerosis de aorta  Cosine Similarity: 0.8956423457669668
calcificación de mama  Cosine Similarity: 0.9110756513664221
callo - lesión  Cosine Similarity: 0.8546052133895455
choque séptico  Cosine Similarity: 0.9255888692511076
cicatriz  Cosine Similarity: 1.0000000016932336
convulsión epiléptica  Cosine Similarity: 0.8516425172222074
cuerpo extraño en estructura corporal  Cosine Similarity: 0.8625446501643084
cálculo renal  Cosine Similarity: 1.0000000106956948
células escamosas atípicas de significación indeterminada en el extendido de Papanicolaou cervica

In [ ]:
#Tiempo de ejecucion: 9 sg
terminos_vinculados = get_terminos_vinculados(model_cc100, terminos_snomedct_problemassaludAH_cc100, 10, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Euclidean Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Euclidean Similarity: 5.31766932519715
accidentes producidos por instrumentos u objetos cortantes y para piercing  Euclidean Similarity: 9.695290535527482
adherencias intestinales con obstrucción  Euclidean Similarity: 6.581675508872707
agresión por instrumentos cortantes y punzantes  Euclidean Similarity: 9.992631494445618
apendicitis  Euclidean Similarity: 0.0
arteriosclerosis obliterante  Euclidean Similarity: 9.86303103502855
ascitis  Euclidean Similarity: 0.0
aterosclerosis aortoilíaca  Euclidean Similarity: 8.957620662073762
aterosclerosis de aorta  Euclidean Similarity: 9.425930817220323
calcificación de mama  Euclidean Similarity: 8.026223142327293
celulitis de la pared abdominal  Euclidean Similarity: 8.54144313030143
cicatriz  Euclidean Similarity: 0.0
cuerpo extraño en estructura corporal  Euclidean Similarity: 9.814464870868743
cálculo renal  Euclidean Similarity: 0.0
células escamosas atípicas de significación indeterminada en el extendido de 

In [ ]:
#Tiempo de ejecucion: 19 sg
terminos_vinculados = get_terminos_vinculados(model_cc100, terminos_snomedct_problemassaludAH_cc100, 0.85, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Cosine Similarity: 0.9555793867210567
adherencias intestinales con obstrucción  Cosine Similarity: 0.8656185613708847
anemia hemolítica  Cosine Similarity: 0.8760345500380488
apendicitis  Cosine Similarity: 0.9999999774699373
ascitis  Cosine Similarity: 0.9999999748623216
asma sin estado asmático  Cosine Similarity: 0.8976650495027991
aterosclerosis aortoilíaca  Cosine Similarity: 0.8894492644873124
aterosclerosis de aorta  Cosine Similarity: 0.8956675265606071
calcificación de mama  Cosine Similarity: 0.8894545811318739
celulitis de la pared abdominal  Cosine Similarity: 0.8711717223807021
cicatriz  Cosine Similarity: 0.9999999848466449
cálculo renal  Cosine Similarity: 0.9999999820622258
derrame pleural  Cosine Similarity: 1.0000000159614437
edema pulmonar  Cosine Similarity: 0.9146033780535122
embolia pulmonar  Cosine Similarity: 0.9199701079906579
enteritis bacteriana  Cosine Similarity: 0.8574589078509391
enteritis viral  Cosine Similarity: 0.90427263

In [ ]:
#Tiempo de ejecucion: 10 sg
terminos_vinculados = get_terminos_vinculados(model_cc300, terminos_snomedct_problemassaludAH_cc300, 10, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Euclidean Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Euclidean Similarity: 5.452100978757312
adherencias intestinales con obstrucción  Euclidean Similarity: 8.053429403891952
apendicitis  Euclidean Similarity: 0.0
ascitis  Euclidean Similarity: 0.0
aterosclerosis de aorta  Euclidean Similarity: 9.578853243148258
calcificación de mama  Euclidean Similarity: 8.35932519631885
celulitis de la pared abdominal  Euclidean Similarity: 9.28467785488095
cicatriz  Euclidean Similarity: 0.0
cálculo renal  Euclidean Similarity: 0.0
células escamosas atípicas de significación indeterminada en el extendido de Papanicolaou cervical  Euclidean Similarity: 9.169378064009488
derrame pleural  Euclidean Similarity: 0.0
edema pulmonar  Euclidean Similarity: 8.968442742225445
embolia pulmonar  Euclidean Similarity: 8.728565852063475
hernia paraumbilical  Euclidean Similarity: 7.327009034043378
hidronefrosis  Euclidean Similarity: 0.0
hidroureteronefrosis  Euclidean Similarity: 7.16593755395356
linfadenopatía  Euclidean Similarity:

In [ ]:
#Tiempo de ejecucion: 19 sg
terminos_vinculados = get_terminos_vinculados(model_cc300, terminos_snomedct_problemassaludAH_cc300, 0.85, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Cosine Similarity: 0.9562747499199189
apendicitis  Cosine Similarity: 0.9999999896109536
ascitis  Cosine Similarity: 0.9999999760695482
aterosclerosis aortoilíaca  Cosine Similarity: 0.87596101164518
aterosclerosis de aorta  Cosine Similarity: 0.8881411039859841
calcificación de mama  Cosine Similarity: 0.8973565127665636
cicatriz  Cosine Similarity: 0.9999999975863367
cálculo renal  Cosine Similarity: 0.9999999983275956
derrame pleural  Cosine Similarity: 1.0000000156039566
edema pulmonar  Cosine Similarity: 0.900826409165184
embolia pulmonar  Cosine Similarity: 0.907290983269277
enteritis viral  Cosine Similarity: 0.8883227422757618
hemangioma hepático  Cosine Similarity: 0.8649057590196244
hernia paraumbilical  Cosine Similarity: 0.9428839026967182
hernia umbilical  Cosine Similarity: 0.9452586750326627
hidronefrosis  Cosine Similarity: 1.000000048407138
lesión hepática  Cosine Similarity: 0.9141457448112001
lesión por presión  Cosine Similarity: 0.8685

In [ ]:
#informe ="el paciente presenta el abdomen distendido"
informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."

In [ ]:
#Tiempo de ejecucion: 9 sg
terminos_vinculados = get_terminos_vinculados(model_cs50, terminos_snomedct_problemassaludAH_cs50, 1.2, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 0.5191199006538469
accidentes producidos por instrumentos u objetos cortantes y para piercing  Similarity: 1.18311368413299
adenoma corticosuprarrenal  Similarity: 1.1927689314830485
adherencias intestinales con obstrucción  Similarity: 0.955139116644531
adherencias peritoneales de la pelvis femenina  Similarity: 1.1740085441128576
amplitud de movimiento articular activo reducido  Similarity: 1.068818043820242
aneurisma de aorta abdominal  Similarity: 0.95509821236321
aneurisma de aorta abdominal en aumento  Similarity: 1.1859622154864704
aneurisma de aorta abdominal sin ruptura  Similarity: 1.1628043407993867
apendicitis  Similarity: 0.0
apendicitis aguda con absceso peritoneal  Similarity: 1.1719071020497462
aplastamiento osteoporótico de vértebra cervical  Similarity: 1.0930553554855602
arteriosclerosis obliterante  Similarity: 1.0316920283419218
ascitis  Similarity: 0.0
asma persistente moderada  Similarity: 1.0475084496247853
ataque isquém

In [ ]:
#Tiempo de ejecucion: 18 sg
terminos_vinculados = get_terminos_vinculados(model_cs50, terminos_snomedct_problemassaludAH_cs50, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 0.9884790723644857
apendicitis  Similarity: 0.9999999289284328
ascitis  Similarity: 1.000000002022122
aterosclerosis de aorta  Similarity: 0.9683588638524672
calcificación de mama  Similarity: 0.9634989978607577
cicatriz  Similarity: 1.0000000686460806
cálculo renal  Similarity: 0.9999999847901402
derrame pleural  Similarity: 0.999999972105954
edema pulmonar  Similarity: 0.9758187998688564
embolia pulmonar  Similarity: 0.9799370222064289
enteritis viral  Similarity: 0.9545609618121784
esputo anormal  Similarity: 0.9516568172144344
hidronefrosis  Similarity: 1.000000028001919
linfadenopatía  Similarity: 1.0000000275007346
lípidos anormales  Similarity: 0.9605421007006723
masa abdominal  Similarity: 0.9894730683652699
masa hepática  Similarity: 0.9743152822078404
masa suprarrenal  Similarity: 0.9999999553725268
obstrucción del intestino delgado  Similarity: 1.0000000601094619
parto normal  Similarity: 0.9510463057524708
soplo abdominal  Similarit

In [ ]:
#Tiempo de ejecucion: 8 sg
terminos_vinculados = get_terminos_vinculados(model_cs100, terminos_snomedct_problemassaludAH_cs100, 1.2, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 0.7088515894150375
aneurisma de aorta abdominal  Similarity: 1.0738537165804998
apendicitis  Similarity: 0.0
ascitis  Similarity: 0.0
aterosclerosis aortoilíaca  Similarity: 1.1109125389461798
aterosclerosis de aorta  Similarity: 0.8353091933860176
calcificación de mama  Similarity: 0.8190229100060146
callo - lesión  Similarity: 0.9303324332578999
cicatriz  Similarity: 0.0
cicatriz queloide  Similarity: 1.122154983783586
cuidado domiciliario del paciente  Similarity: 1.1198480022944988
cálculo renal  Similarity: 0.0
células escamosas atípicas de significación indeterminada en el extendido de Papanicolaou cervical  Similarity: 1.1665779935047944
derrame pleural  Similarity: 0.0
edema pulmonar  Similarity: 0.7381361373589997
embolia pulmonar  Similarity: 1.0626056709928173
engrosamiento del tallo hipofisario  Similarity: 1.1508224704451775
enteritis viral  Similarity: 1.0885392786368482
esputo anormal  Similarity: 1.1914847053641255
estenosis deg

In [ ]:
#Tiempo de ejecucion: 19 sg
terminos_vinculados = get_terminos_vinculados(model_cs100, terminos_snomedct_problemassaludAH_cs100, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 0.9786927210486015
apendicitis  Similarity: 1.000000047841527
ascitis  Similarity: 1.0000000231934383
aterosclerosis de aorta  Similarity: 0.9674901046525459
calcificación de mama  Similarity: 0.9630764690585242
cicatriz  Similarity: 0.999999982111321
cálculo renal  Similarity: 1.0000000208821584
derrame pleural  Similarity: 0.99999994684797
edema pulmonar  Similarity: 0.9781743648019987
hidronefrosis  Similarity: 1.000000004821988
linfadenopatía  Similarity: 1.0000000204927266
lípidos anormales  Similarity: 0.9603839480889819
masa abdominal  Similarity: 0.986389819058891
masa hepática  Similarity: 0.9748264146526643
masa suprarrenal  Similarity: 0.9999999607581544
obstrucción del intestino delgado  Similarity: 0.9999999839856961
soplo abdominal  Similarity: 0.9908591078665472
voz anormal  Similarity: 0.9780232254223178


In [ ]:
#Tiempo de ejecucion: 10 sg
terminos_vinculados = get_terminos_vinculados(model_cs300, terminos_snomedct_problemassaludAH_cs300, 1.2, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 0.9233627689449166
apendicitis  Similarity: 0.0
ascitis  Similarity: 0.0
aterosclerosis de aorta  Similarity: 1.0270846043068973
calcificación de mama  Similarity: 1.035769935749844
cicatriz  Similarity: 0.0
cálculo renal  Similarity: 0.0
derrame pleural  Similarity: 0.0
edema pulmonar  Similarity: 0.9805518898840251
hernia paraumbilical  Similarity: 1.0406125166549651
hidronefrosis  Similarity: 0.0
hidroureteronefrosis  Similarity: 1.039189602609959
linfadenopatía  Similarity: 0.0
masa abdominal  Similarity: 0.7186124738374491
masa hepática  Similarity: 1.1760545645068547
masa suprarrenal  Similarity: 0.0
obstrucción del intestino delgado  Similarity: 0.0
soplo abdominal  Similarity: 0.5677505878332866
voz anormal  Similarity: 1.0313730251002204


In [ ]:
#Tiempo de ejecucion: 19 sg
terminos_vinculados = get_terminos_vinculados(model_cs300, terminos_snomedct_problemassaludAH_cs300, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 0.9772876887830823
apendicitis  Similarity: 0.9999999772982147
ascitis  Similarity: 1.0000000340799877
aterosclerosis de aorta  Similarity: 0.9648783250524904
calcificación de mama  Similarity: 0.9587587180805336
cicatriz  Similarity: 1.000000003340891
cálculo renal  Similarity: 0.9999999900897357
derrame pleural  Similarity: 0.9999999937720159
edema pulmonar  Similarity: 0.9767364056077397
hidronefrosis  Similarity: 0.9999999705395328
linfadenopatía  Similarity: 1.0000000248003937
lípidos anormales  Similarity: 0.9543572264979956
masa abdominal  Similarity: 0.9860515295477978
masa hepática  Similarity: 0.9580333264366948
masa suprarrenal  Similarity: 1.0000000312131383
obstrucción del intestino delgado  Similarity: 0.999999991393781
soplo abdominal  Similarity: 0.9911524021151514
voz anormal  Similarity: 0.9763272979481873
